In [ ]:
#Calibrate camera
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt 

img = Image.open("photodata/drawer0.jpg")
print(f"Image is {img.width} x {img.height}")
points = np.array(
    [
        [314.0, 866.0],  # [0]
        [290.0, 1388.0],  # [1]
        [677.0, 797.0],  # [2]
        [463.0, 1583.0],  # [3]
        [523.0, 1012.0],  # [4]
        [905.0, 914.0],  # [5]
    ]
)

# Visualize image & annotated points
fig, ax = plt.subplots(figsize=(8, 10))
ax.imshow(img)
ax.scatter(points[:, 0], points[:, 1], color="white", marker="x")
for i in range(len(points)):
    ax.annotate(
        f"points[{i}]",
        points[i] + np.array([15.0, 5.0]),
        color="white",
        backgroundcolor=(0, 0, 0, 0.15),
        zorder=0.1,
    )

In [ ]:
# Select points used to compute each vanishing point
#
# Each `v*_indices` list should contain four integers, corresponding to
# indices into the `points` array; the first two ints define one line and
# the second two define another line.
from calibrate import intersection_from_lines
        
v0_indices = None
v1_indices = None
v2_indices = None

v0_indices = [0, 1, 3, 4]
v1_indices = [0, 2, 4, 5]
v2_indices = [0, 4, 2, 5]

# Compute vanishing points
v = np.zeros((3, 2))
v[:, :2] = np.array(
    [
        intersection_from_lines(*points[v0_indices]),
        intersection_from_lines(*points[v1_indices]),
        intersection_from_lines(*points[v2_indices]),
    ]
)


In [ ]:
from calibrate import optical_center_from_vanishing_points

optical_center = optical_center_from_vanishing_points(v[0], v[1], v[2],)
c_x, c_y = optical_center

In [ ]:
from calibrate import focal_length_from_two_vanishing_points

# If your implementation is correct, these should all be ~the same
f = focal_length_from_two_vanishing_points(v[0], v[1], optical_center)
print(f"Focal length from v0, v1: {f}")
f = focal_length_from_two_vanishing_points(v[1], v[2], optical_center)
print(f"Focal length from v1, v2: {f}")
f = focal_length_from_two_vanishing_points(v[0], v[2], optical_center)
print(f"Focal length from v0, v2: {f}")

In [ ]:
#Identifying the board coordinates

board_z_distance = 384 #(inches)
board_coordinates = [882, 925]
img = Image.open("photodata/divingframe124.jpg")
fig, ax = plt.subplots(figsize=(8, 10))
ax.imshow(img)
ax.scatter(board_coordinates[0], board_coordinates[1], color="red", marker="x")

In [ ]:
#Get Keypoints / Box coordinates
#POSE ESTIMATION SAVING TO ARRAY
from poseEstimation import estimate_pose


data_frames, keypoints, boxes = estimate_pose("videos/IMG_2478.mov")

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt 

In [ ]:
import numpy as np
import cv2
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 100

fig, ax = plt.subplots(figsize=(10, 8)) 

im = ax.imshow(np.zeros(data_frames[0].shape, dtype=np.uint8))

def init():
    im.set_data(np.zeros(data_frames[0].shape, dtype=np.uint8))
    return (im,)

def update(frame_number):
    rgb_image = cv2.cvtColor(data_frames[frame_number], cv2.COLOR_BGR2RGB)
    im.set_data(rgb_image)
    return (im,)

# Create the animation
ani_pose = FuncAnimation(fig, update, frames=len(data_frames), init_func=init, interval=50, blit=True)

In [ ]:
HTML(ani_pose.to_jshtml())

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt 


In [ ]:
#Predict Labels
import torch
import torch.nn as nn

class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(38, 30)  
        self.act1 = nn.ReLU()             
        self.hidden2 = nn.Linear(30, 20)  
        self.act2 = nn.ReLU()             
        self.output = nn.Linear(20, 3)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))    
        x = self.act2(self.hidden2(x))    
        x = self.output(x)       
        return x
    
model = Multiclass()  # Re-create the model structure
model.load_state_dict(torch.load('model_best_weights.pth'))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import cv2
from IPython.display import HTML
from projection import diver_height_and_distance

# Assuming data_frames is your list of frames in BGR format

plt.rcParams['animation.embed_limit'] = 100

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(np.zeros(data_frames[0].shape, dtype=np.uint8))

# Placeholder prediction display text
prediction_text = ax.text(10, 10, '', fontsize=18, color='black')
height_text = ax.text(10, 10, '', fontsize=18, color='black')
distance_text = ax.text(10, 10, '', fontsize=18, color='black')



def get_model_prediction(keypoints, boxes):
    positions = ["Not in air", "Straight", "Pike"]
    # Placeholder for your model's prediction logic
    # Use the keypoints as input to your model and return the prediction
    if keypoints.size()[1] == 17:
        currKeypoints = keypoints[0]
        currBoxes = boxes
        combined = torch.cat((torch.flatten(currKeypoints), currBoxes))
        prediction = model(combined)
        prediction = torch.argmax(prediction)
        return positions[prediction]
    return "No detection"

def get_height_distance(boxes, f, c_x, c_y, board_z_distance, board_coordinates):
    x = int(boxes[0] + boxes[2]) / 2
    y = int(boxes[1] + boxes[3]) / 2
    diver_coordinates = [x, y]
    height, distance = diver_height_and_distance(board_coordinates, diver_coordinates, f, c_x, c_y, board_z_distance)
    height = "height: " + str(round(height / 12, 1)) + " ft"
    distance = "distance: " + str(round(distance / 12, 1)) + " ft"
    return height, distance

def init():
    im.set_data(np.zeros(data_frames[0].shape, dtype=np.uint8))
    prediction_text.set_text('')
    return (im, prediction_text)

def update(frame_number):
    rgb_image = cv2.cvtColor(data_frames[frame_number], cv2.COLOR_BGR2RGB)
    im.set_data(rgb_image)
    
    # Get model prediction for current frame
    prediction = get_model_prediction(keypoints[frame_number], boxes[frame_number])
    prediction_text.set_text(prediction)
    height, distance = get_height_distance(boxes[frame_number], f, c_x, c_y, board_z_distance, board_coordinates)
    height_text.set_text(height)
    distance_text.set_text(distance)
    
    # Optionally, adjust the position of your text if needed
    prediction_text.set_position((100, 100))
    height_text.set_position((100, 200))
    distance_text.set_position((100, 300))
    
    
    return (im, prediction_text)

ani_pred = FuncAnimation(fig, update, frames=len(data_frames), init_func=init, interval=50, blit=True)

In [ ]:
HTML(ani_pred.to_jshtml())